# Procesamiento de climatologías - agrupación por mes

Este _notebook_ contiene programas en Python para el procesamiento de climatologías, agrupándolas por año. Una climatología se define como un conjunto de medidas estadísticas que resumen series temporales de variables climáticas como precipitación, temperatura, radiación solar y velocidad del viento, entre otras.

**Bibliotecas**

In [1]:
import os

import rasterio

from shapely.geometry import Point

import pandas as pd
import geopandas as gpd

## Entrada

In [2]:
# Directorio de datos de entrada
INPUT_DATA_DIR = "/home/mfvargas/procesamiento-climatologias-datos/chelsa-recortados-cr"

# Directorio de datos de salida
OUTPUT_DATA_DIR = "/home/mfvargas/procesamiento-climatologias-datos/chelsa-recortados-cr-agrupados-mes"

# Año inicial
START_MONTH = 1

# Año final
END_MONTH = 2

# Valor de NO DATA
NO_DATA_VALUE = 65535

## Procesamiento

In [4]:
for month in range(START_MONTH, END_MONTH + 1):
    print("month:", month)

    # DataFrame
    centros_gdf = gpd.GeoDataFrame(columns=['geometry', 'x', 'y'], crs={"init": "epsg:4326"})

    # Contador de archivos
    f = 0

    # Recorrido de los archivos del directorio de entrada
    files = os.listdir(INPUT_DATA_DIR)
    files.sort()
    for file in files:
        anio = file[12:16]
        mes = file[17:19]
        
        print("anio:", anio, "mes:", mes, "str(month)", str(month))

        if mes != "0" + str(month):
            continue
            
        #if mes not in ('01', '02'):
        #    continue            

        print("Año:", anio, "Mes:", mes)
        centros_gdf['c_' + anio + '_' + mes] = 65535

        dataset = rasterio.open(INPUT_DATA_DIR + "/" + file)
        print("Archivo:", file, "Ancho:", dataset.width, "Largo:", dataset.height)

        # Recuperación de los valores de la banda 1 en un arreglo
        band = dataset.read(1)

        # Recorrido de las filas y columnas de la banda
        i = 0
        t = 0
        for row in band:
            print("i:", i)
            j = 0
            for value in row:
                if value != NO_DATA_VALUE:
                    # coordinates = transform.xy(dataset.transform, j, i, offset='center')                    
                    # coordinates = dataset.transform * (j, i)
                    coordinates = dataset.xy(i, j)
                    # print(coordinates)
                    x = coordinates[0]
                    y = coordinates[1]
                    point = Point(coordinates)
                    # print(t, i, j, x, y, point, value)
                    if (f == 0):
                        centros_gdf.loc[t] = [point] + [x] + [y] + [value]
                    else:
                        centros_gdf.loc[[t], ['c_' + anio + '_' + mes]] = value
                    t += 1
                j += 1

            i += 1
        f += 1

    # Columna de la suma
    centros_gdf['suma'] = centros_gdf.iloc[:,3:].sum(axis=1)    

    # Columna del promedio
    centros_gdf['promedio'] = centros_gdf.iloc[:,3:(len(centros_gdf.columns)-1)].mean(axis=1)
    
    # Archivo CSV de salida
    centros_gdf.to_file(OUTPUT_DATA_DIR + "/" + "CHELSA_"+str(month)+".csv", driver="CSV")
    
    # Archivo SHP de salida
    centros_gdf.to_file(OUTPUT_DATA_DIR + "/" + "CHELSA_"+str(month)+".shp")

month: 1
anio: 1979 mes: 01 str(month) 1
Año: 1979 Mes: 01
Archivo: CHELSA_prec_1979_01_V1.2.1.tif Ancho: 411 Largo: 382
i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48


KeyboardInterrupt: 

In [ ]:
centros_gdf